<a href="https://colab.research.google.com/github/yi1982/try/blob/master/data_selection_and_feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import pandas
import pandas as pd
#import numpy
import numpy as np
#import matplotlib
import matplotlib.pyplot as plt
#import seaborn
import seaborn as sns
#import scipy.stats
import scipy.stats as stats
#import reduce
from functools import reduce
import random
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#set path and datatype
path = '/content/drive/My Drive/Colab Notebooks/data/'

data_dtype = {
    'order_id': np.int32,
    'user_id': np.int32,
    'eval_set': 'category',
    'order_number': np.int8,
    'order_dow': np.int8,
    'order_hour_of_day': np.int8,
    'days_since_prior_order': np.float16,
    'product_id': np.int32,
    'add_to_cart_order': np.int8,
    'reordered': np.int8
}

In [ ]:
aisles = pd.read_csv(path + 'aisles.csv')
departments = pd.read_csv(path + 'departments.csv')
products = pd.read_csv(path + 'products.csv')

In [ ]:
orders = pd.read_csv(path + 'orders.csv',dtype=data_dtype)

In [ ]:
orders.isna().sum() # one NA for each user 206209 NA,keep NA for now

order_id                       0
user_id                        0
eval_set                       0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

In [ ]:
order_products_train = pd.read_csv(path + 'order_products__train.csv',dtype=data_dtype)

In [ ]:
order_products_prior = pd.read_csv(path + 'order_products__prior.csv',dtype=data_dtype)

In [ ]:
#merge orders[orders['eval_set'] == 'prior'] and order_products_prior to get products ordering information in prior orders
prior_products = pd.merge(orders[orders['eval_set'] == 'prior'],
                    order_products_prior,on='order_id')
prior_products.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0


In [ ]:
prior_products.shape

(32434489, 10)

32434489 user_products pair in the prior_products

In [ ]:
prior_products.groupby('reordered')['order_id'].count().to_frame('num_of_count')

,num_of_count
reordered,
0,13307953
1,19126536


In [ ]:
ratio_original_prior = 13307953/19126536
ratio_original_prior

0.6957847986692415

reordered 0, 1 ratio =0.696

In [ ]:
#merge orders[orders['eval_set'] == 'train'] and order_products_train to get products ordering information in the last orders
train_products = pd.merge(orders[orders['eval_set'] == 'train'],
                    order_products_train,on='order_id')
train_products.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,1187899,1,train,11,4,8,14.0,196,1,1
1,1187899,1,train,11,4,8,14.0,25133,2,1
2,1187899,1,train,11,4,8,14.0,38928,3,1
3,1187899,1,train,11,4,8,14.0,26405,4,1
4,1187899,1,train,11,4,8,14.0,39657,5,1


In [ ]:
train_products.groupby('reordered')['order_id'].count().to_frame('count')

,count
reordered,
0,555793
1,828824


In [ ]:
ratio_original_train = 555793/828824
ratio_original_train

0.6705802438153335

reordered 0, 1 ratio =0.671

In [ ]:
# since there are too many user product pairs, after feature engineening, the model training would be very expensive. so I will only use 1/6 of users' data for this project
user_id = list(orders[orders['eval_set'] == 'train']['user_id'])
num = len(user_id)//6

In [ ]:
random.seed(5)
user_selected = random.sample(user_id, num)

split into training and testing set

In [ ]:
# Split the user_selected into training and testing set.
user_id_train, user_id_test = train_test_split(user_selected,test_size=0.2,random_state=5)

print(len(user_selected),len(user_id_train),len(user_id_test))

21868 17494 4374


In [ ]:
train_products[train_products['user_id'].isin(user_id_test)].groupby('reordered')['order_id'].count().to_frame('count')

,count
reordered,
0,18191
1,28213


In [ ]:
ratio_new_train = 18191/28213
ratio_new_train

0.6447736858894836

In [ ]:
prior_products[prior_products['user_id'].isin(user_id_test)].groupby('reordered')['order_id'].count().to_frame('count')

,count
reordered,
0,285493
1,430025


In [ ]:
ratio_new_prior = 285493/430025
ratio_new_prior

0.6638986105458985

reordered 0.1 ratio doesnot change much for both prior and train data

In [ ]:
# Split DataFrames based on user_id
X_products_train = prior_products[prior_products['user_id'].isin(user_id_train)]
X_products_test = prior_products[prior_products['user_id'].isin(user_id_test)]

In [ ]:
X_orders_train = orders[orders['user_id'].isin(user_id_train)]
X_orders_test = orders[orders['user_id'].isin(user_id_test)]
X_orders_train = X_orders_train[X_orders_train['eval_set'] == 'prior']
X_orders_test = X_orders_test[X_orders_test['eval_set'] == 'prior']

# **target**

In [ ]:
# select all the user-product pair in prior order for training ang testing set
user_product_pair_train = X_products_train.groupby(['user_id','product_id'])['order_number'].agg('min').to_frame('in_train_set').reset_index()
user_product_pair_train['in_train_set'] = True

In [ ]:
user_product_pair_test = X_products_test.groupby(['user_id','product_id'])['order_number'].agg('min').to_frame('in_test_set').reset_index()
user_product_pair_test['in_test_set'] = True

In [ ]:
#merge dataframe to get the last order products information for original train set, and do train test split based on user_id
last_orders = pd.merge(order_products_train, orders[orders['eval_set'] == 'train'], how='left',on='order_id')
y_train = last_orders[last_orders['user_id'].isin(user_id_train)]
y_test = last_orders[last_orders['user_id'].isin(user_id_test)]

In [ ]:
#merge with the user_product pairs in training set 
y_train_merge = pd.merge(user_product_pair_train,y_train[['user_id','product_id','reordered']],how='left',on=['user_id','product_id'])

In [ ]:
#merge with the user_product pairs in testing set
y_test_merge = pd.merge(user_product_pair_test,y_test[['user_id','product_id','reordered']],how='left',on=['user_id','product_id'])

In [ ]:
# fill na with 0 to indicate not reordered
y_train_merge.fillna(0,inplace=True)

In [ ]:
y_test_merge.fillna(0,inplace=True)

In [ ]:
y_train_merge.drop(columns ='in_train_set',axis=1,inplace=True)

In [ ]:
y_test_merge.drop(columns ='in_test_set',axis=1,inplace=True)

In [ ]:
y_train_merge.groupby('reordered').count()

,user_id,product_id
reordered,,
0.0,1024609,1024609
1.0,110461,110461


In [ ]:
y_test_merge.groupby('reordered').count()

,user_id,product_id
reordered,,
0.0,257280,257280
1.0,28213,28213


In [ ]:
# save to csv and upload to google drive
y_train_merge.to_csv('y_train.csv')
!cp y_train.csv "drive/My Drive//Colab Notebooks/data/"

In [ ]:
y_test_merge.to_csv('y_test.csv')
!cp y_test.csv "drive/My Drive//Colab Notebooks/data/"

# **feature** **engineering**

user related features

In [ ]:
#store user related features in a dictionary
user_features_train = {} 
user_features_test = {}

In [ ]:
# total number of orders each user has placed

def num_of_orders(df):
  return df.groupby('user_id')['order_number'].count().to_frame('u_num_of_orders').reset_index()

user_features_train['num_of_orders'] = num_of_orders(X_orders_train)
user_features_test['num_of_orders'] = num_of_orders(X_orders_test)

In [ ]:
#total number of products each user has purchased

def num_of_products(df):
  return df.groupby(by=['user_id'])['product_id'].count().to_frame('u_num_of_products').reset_index()

user_features_train['num_of_products'] = num_of_products(X_products_train)
user_features_test['num_of_products'] = num_of_products(X_products_test)

In [ ]:
#average number of products each user has purchased per order
def products_mean(df):
  return df.groupby(['user_id','order_number'])['add_to_cart_order'].max().groupby('user_id').mean().to_frame('u_products_mean').reset_index()

user_features_train['products_mean'] = products_mean(X_products_train)
user_features_test['products_mean'] = products_mean(X_products_test)

In [ ]:
#std of products each user has purchased per order
def products_std(df):
  return df.groupby(['user_id','order_number'])['add_to_cart_order'].max().groupby('user_id').std().to_frame('u_products_std').reset_index()

user_features_train['products_std'] = products_std(X_products_train) 
user_features_test['products_std'] = products_std(X_products_test)

In [ ]:
#number of unique products each user has purchased
def unique_products(df):
  return df.groupby('user_id')['product_id'].nunique().to_frame('u_unique_products').reset_index()

user_features_train['unique_products'] = unique_products(X_products_train)
user_features_test['unique_products'] = unique_products(X_products_test)

In [ ]:
#number of total products each user has purchased which are reordered
def num_of_reordered_products(df):
  return df.groupby('user_id')['reordered'].sum().to_frame('u_num_of_reordered_products').reset_index()

user_features_train['num_of_reordered_products'] = num_of_reordered_products(X_products_train) 
user_features_test['num_of_reordered_products'] = num_of_reordered_products(X_products_test)

In [ ]:
#average number of reordered products each user has purchased per order
def reordered_mean(df):
  return df.groupby(['user_id','order_number'])['reordered'].sum().groupby('user_id').mean().to_frame('u_reordered_mean').reset_index()

user_features_train['reordered_mean'] = reordered_mean(X_products_train) 
user_features_test['reordered_mean'] = reordered_mean(X_products_test) 

In [ ]:
#std of reordered products each user has purchased per order
def reordered_std(df):
  return df.groupby(['user_id','order_number'])['reordered'].sum().groupby('user_id').std().to_frame('u_reordered_std').reset_index()

user_features_train['reordered_std'] = reordered_std(X_products_train)
user_features_test['reordered_std'] = reordered_std(X_products_test) 

In [ ]:
#mean of order_dow for each user
def order_dow_mean(df):
  return df.groupby(['user_id','order_number'])['order_dow'].agg('min').groupby('user_id').mean().to_frame('u_order_dow_mean').reset_index()

user_features_train['order_dow_mean'] = order_dow_mean(X_products_train)
user_features_test['order_dow_mean'] = order_dow_mean(X_products_test) 

In [ ]:
#std of order_dow for each user
def order_dow_std(df):
  return df.groupby(['user_id','order_number'])['order_dow'].agg('min').groupby('user_id').std().to_frame('u_order_dow_std').reset_index()

user_features_train['order_dow_std'] = order_dow_std(X_products_train)
user_features_test['order_dow_std'] = order_dow_std(X_products_test)

In [ ]:
#mean of order_hour_of_day for each user
def order_hour_of_day_mean(df):
  return df.groupby(['user_id','order_number'])['order_hour_of_day'].agg('min').groupby('user_id').mean().to_frame('u_order_hour_of_day_mean').reset_index()

user_features_train['order_hour_of_day_mean'] = order_hour_of_day_mean(X_products_train)
user_features_test['order_hour_of_day_mean'] = order_hour_of_day_mean(X_products_test) 

In [ ]:
#std of order_hour_of_day for each user
def order_hour_of_day_std(df):
  return df.groupby(['user_id','order_number'])['order_hour_of_day'].agg('min').groupby('user_id').std().to_frame('u_order_hour_of_day_std').reset_index()

user_features_train['order_hour_of_day_std'] = order_hour_of_day_std(X_products_train)
user_features_test['order_hour_of_day_std'] = order_hour_of_day_std(X_products_test) 

In [ ]:
#mean of days_since_prior_order for each user 
def days_between_orders_mean(df):
  return df[~df['days_since_prior_order'].isna()].groupby(['user_id','order_number'])['days_since_prior_order'].agg('min').groupby('user_id').mean().to_frame('u_days_between_orders_mean').reset_index()

user_features_train['days_between_orders_mean'] = days_between_orders_mean(X_products_train) 
user_features_test['days_between_orders_mean'] = days_between_orders_mean(X_products_test) 

In [ ]:
#std of days_since_prior_order for each user 
def days_between_orders_std(df):
  return df[~df['days_since_prior_order'].isna()].groupby(['user_id','order_number'])['days_since_prior_order'].agg('min').groupby('user_id').std().to_frame('u_days_between_orders_std').reset_index()

user_features_train['days_between_orders_std'] = days_between_orders_std(X_products_train) 
user_features_test['days_between_orders_std'] = days_between_orders_std(X_products_test) 

In [ ]:
#number of the orders containing reordered items for each users
def num_of_orders_containg_reorder(df):
    return df.groupby(['user_id','order_number'])['reordered'].agg('max').groupby('user_id').sum().to_frame('u_num_of_orders_containg_reorder').reset_index()

user_features_train['num_of_orders_containg_reorder'] = num_of_orders_containg_reorder(X_products_train)
user_features_test['num_of_orders_containg_reorder'] = num_of_orders_containg_reorder(X_products_test) 

In [ ]:
# merge features together
df_user_features_train = reduce(lambda  left,right: pd.merge(left,right,on=['user_id'],
                                            how='left'), [feature for key, feature in user_features_train.items()])
df_user_features_test = reduce(lambda  left,right: pd.merge(left,right,on=['user_id'],
                                            how='left'), [feature for key, feature in user_features_test.items()])

In [ ]:
# proportion of products each user has purchased which are reordered
df_user_features_train['u_reordered_products_ratio']=df_user_features_train['u_num_of_reordered_products']/df_user_features_train['u_num_of_products']
# proportion of orders each user has placed which has reordered products
df_user_features_train['u_reordered_order_ratio']=df_user_features_train['u_num_of_orders_containg_reorder']/df_user_features_train['u_num_of_orders']

df_user_features_test['u_reordered_products_ratio']=df_user_features_test['u_num_of_reordered_products']/df_user_features_test['u_num_of_products']
df_user_features_test['u_reordered_order_ratio']=df_user_features_test['u_num_of_orders_containg_reorder']/df_user_features_test['u_num_of_orders']

df_user_features_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17494 entries, 0 to 17493
Data columns (total 18 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   user_id                           17494 non-null  int64  
 1   u_num_of_orders                   17494 non-null  int64  
 2   u_num_of_products                 17494 non-null  int64  
 3   u_products_mean                   17494 non-null  float64
 4   u_products_std                    17494 non-null  float64
 5   u_unique_products                 17494 non-null  int64  
 6   u_num_of_reordered_products       17494 non-null  float64
 7   u_reordered_mean                  17494 non-null  float64
 8   u_reordered_std                   17494 non-null  float64
 9   u_order_dow_mean                  17494 non-null  float64
 10  u_order_dow_std                   17494 non-null  float64
 11  u_order_hour_of_day_mean          17494 non-null  float64
 12  u_or

In [ ]:
# save to csv and upload to google drive
df_user_features_train.to_csv('user_features_train.csv')
!cp user_features_train.csv "drive/My Drive//Colab Notebooks/data/"

In [ ]:
# save to csv and upload to google drive
df_user_features_test.to_csv('user_features_test.csv')
!cp user_features_test.csv "drive/My Drive//Colab Notebooks/data/"

# **product related features**

In [ ]:
#store product related features in a dictionary
product_features_train = {} 
product_features_test = {}

In [ ]:
#number of times the product has been purchased
def p_num_of_orders(df):
  return df.groupby('product_id')['user_id'].count().to_frame('p_num_of_orders').reset_index()

product_features_train['p_num_of_orders'] = p_num_of_orders(X_products_train)
product_features_test['p_num_of_orders'] = p_num_of_orders(X_products_test)

In [ ]:
#number of users who have purchased the product
def p_num_of_users(df):
  return df.groupby('product_id')['user_id'].nunique().to_frame('p_num_of_users').reset_index()

product_features_train['p_num_of_users'] = p_num_of_users(X_products_train)
product_features_test['p_num_of_users'] = p_num_of_users(X_products_test)

In [ ]:
#reorder ratio
def p_reorder_ratio(df):
  return df.groupby('product_id')['reordered'].mean().to_frame('p_reorder_ratio').reset_index()

product_features_train['p_reorder_ratio'] = p_reorder_ratio(X_products_train)
product_features_test['p_reorder_ratio'] = p_reorder_ratio(X_products_test)

In [ ]:
#mean order_hour_of_day
def p_order_hour_of_day_mean(df):
  return df.groupby('product_id')['order_hour_of_day'].mean().to_frame('p_order_hour_of_day_mean').reset_index()

product_features_train['p_order_hour_of_day_mean'] = p_order_hour_of_day_mean(X_products_train)
product_features_test['p_order_hour_of_day_mean'] = p_order_hour_of_day_mean(X_products_test)

In [ ]:
#std order_hour_of_day
def p_order_hour_of_day_std(df):
  return df.groupby('product_id')['order_hour_of_day'].std().to_frame('p_order_hour_of_day_std').reset_index()
  
#some products have only been ordered once, so the std is NA, fillNA with 0

product_features_train['p_order_hour_of_day_std'] = p_order_hour_of_day_std(X_products_train).fillna(0)
product_features_test['p_order_hour_of_day_Std'] = p_order_hour_of_day_std(X_products_test).fillna(0)

In [ ]:
#average order_dow
def p_order_dow_mean(df):
  return df.groupby('product_id')['order_dow'].mean().to_frame('p_order_dow_mean').reset_index()

product_features_train['p_order_dow_mean'] = p_order_dow_mean(X_products_train)
product_features_test['p_order_dow_mean'] = p_order_dow_mean(X_products_test)

In [ ]:
#std order_dow
def p_order_dow_std(df):
  return df.groupby('product_id')['order_dow'].std().to_frame('p_order_dow_std').reset_index()

#p_order_dow_std_train = p_order_dow_std(X_products_train)
#p_order_dow_std_test = p_order_dow_std(X_products_test)

product_features_train['p_order_dow_std'] = p_order_dow_std(X_products_train).fillna(0)
product_features_test['p_order_dow_std'] = p_order_dow_std(X_products_test).fillna(0)

In [ ]:
#average add_to_cart_order of each product
def p_add_to_cart_order_mean(df):
  return df.groupby('product_id')['add_to_cart_order'].mean().to_frame('p_add_to_cart_order_mean').reset_index()

product_features_train['p_add_to_cart_order_mean'] = p_add_to_cart_order_mean(X_products_train)
product_features_test['p_add_to_cart_order_mean'] = p_add_to_cart_order_mean(X_products_test)

In [ ]:
#std of add_to_cart_order of each product
def p_add_to_cart_order_std(df):
  return df.groupby('product_id')['add_to_cart_order'].std().to_frame('p_add_to_cart_order_std').reset_index()

product_features_train['p_add_to_cart_order_std'] = p_add_to_cart_order_std(X_products_train).fillna(0)
product_features_test['p_add_to_cart_order_std'] = p_add_to_cart_order_std(X_products_test).fillna(0)

In [ ]:
#average days_between_orders of each product
def p_days_between_orders_mean(df):
#  return df[~df['days_since_prior_order'].isna()].groupby(['product_id','order_number'])['days_since_prior_order'].agg('min').groupby('product_id').mean().to_frame('p_days_between_orders_mean').reset_index()
  return df[~df['days_since_prior_order'].isna()].groupby(['product_id'])['days_since_prior_order'].mean().to_frame('p_days_between_orders_mean').reset_index()

#some products have only been ordered in the first order of users, so no days_between_orders_mean, fillna will infinity or longest interval after merge?
product_features_train['p_days_between_orders_mean'] = p_days_between_orders_mean(X_products_train).fillna(np.Inf)
product_features_test['p_days_between_orders_mean'] = p_days_between_orders_mean(X_products_test).fillna(np.Inf)

In [ ]:
#std of days_between_orders of each product
def p_days_between_orders_std(df):
  return df[~df['days_since_prior_order'].isna()].groupby(['product_id'])['days_since_prior_order'].std().to_frame('p_days_between_orders_std').reset_index()

product_features_train['p_days_between_orders_std'] = p_days_between_orders_std(X_products_train).fillna(0)
product_features_test['p_days_between_orders_std'] = p_days_between_orders_std(X_products_train).fillna(0)

In [ ]:
#average number of products in the same order of each product and std
products_info = X_products_train[['order_id','product_id','add_to_cart_order']].copy(deep=True)
order_size = products_info.groupby('order_id')['add_to_cart_order'].max()
products_order_size = pd.merge(products_info[['order_id','product_id']],order_size,how='left',on='order_id')

product_features_train['p_num_of_products_mean'] = products_order_size.groupby('product_id')['add_to_cart_order'].mean().to_frame('p_num_of_products_mean').reset_index()
product_features_train['p_num_of_products_std'] = products_order_size.groupby('product_id')['add_to_cart_order'].std().to_frame('p_num_of_products_std').reset_index().fillna(0)

In [ ]:
#average number of products in the same order of each product and std test set
products_info_test = X_products_test[['order_id','product_id','add_to_cart_order']].copy(deep=True)
order_size_test = products_info_test.groupby('order_id')['add_to_cart_order'].max()
products_order_size_test = pd.merge(products_info_test[['order_id','product_id']],order_size_test,how='left',on='order_id')

product_features_test['p_num_of_products_mean'] = products_order_size_test.groupby('product_id')['add_to_cart_order'].mean().to_frame('p_num_of_products_mean').reset_index()
product_features_test['p_num_of_products_std'] = products_order_size_test.groupby('product_id')['add_to_cart_order'].std().to_frame('p_num_of_products_std').reset_index().fillna(0)

In [ ]:
df_product_features_train = reduce(lambda  left,right: pd.merge(left,right,on=['product_id'],
                                            how='left'), [feature for key, feature in product_features_train.items()])
df_product_features_test = reduce(lambda  left,right: pd.merge(left,right,on=['product_id'],
                                            how='left'), [feature for key, feature in product_features_test.items()])

In [ ]:
df_product_features_train.head()
df_product_features_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27458 entries, 0 to 27457
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  27458 non-null  int64  
 1   p_num_of_orders             27458 non-null  int64  
 2   p_num_of_users              27458 non-null  int64  
 3   p_reorder_ratio             27458 non-null  float64
 4   p_order_hour_of_day_mean    27458 non-null  float64
 5   p_order_hour_of_day_std     27458 non-null  float64
 6   p_order_dow_mean            27458 non-null  float64
 7   p_order_dow_std             27458 non-null  float64
 8   p_add_to_cart_order_mean    27458 non-null  float64
 9   p_add_to_cart_order_std     27458 non-null  float64
 10  p_days_between_orders_mean  26771 non-null  float16
 11  p_days_between_orders_std   25571 non-null  float16
 12  p_num_of_products_mean      27458 non-null  float64
 13  p_num_of_products_std       274

In [ ]:
df_product_features_train['p_days_between_orders_mean'].fillna(np.Inf,inplace=True)
df_product_features_test['p_days_between_orders_mean'].fillna(np.Inf,inplace=True)
df_product_features_train['p_days_between_orders_std'].fillna(0,inplace=True)
df_product_features_test['p_days_between_orders_std'].fillna(0,inplace=True)
df_product_features_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40029 entries, 0 to 40028
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  40029 non-null  int64  
 1   p_num_of_orders             40029 non-null  int64  
 2   p_num_of_users              40029 non-null  int64  
 3   p_reorder_ratio             40029 non-null  float64
 4   p_order_hour_of_day_mean    40029 non-null  float64
 5   p_order_hour_of_day_std     40029 non-null  float64
 6   p_order_dow_mean            40029 non-null  float64
 7   p_order_dow_std             40029 non-null  float64
 8   p_add_to_cart_order_mean    40029 non-null  float64
 9   p_add_to_cart_order_std     40029 non-null  float64
 10  p_days_between_orders_mean  40029 non-null  float16
 11  p_days_between_orders_std   40029 non-null  float16
 12  p_num_of_products_mean      40029 non-null  float64
 13  p_num_of_products_std       400

In [ ]:
# save to csv and upload to google drive
df_product_features_train.to_csv('product_features_train.csv')
df_product_features_test.to_csv('product_features_test.csv')

In [ ]:
!cp product_features_train.csv "drive/My Drive//Colab Notebooks/data/"
!cp product_features_test.csv "drive/My Drive//Colab Notebooks/data/"

# **User** **X** **Product** **feature**

In [ ]:
# store user product interaction features in dictionary
UxP_features_train = {}
UxP_features_test = {}

In [ ]:
#number of times the user has purchased the product
def UxP_num_of_orders(df):
  return df.groupby(['user_id','product_id'])['order_id'].count().to_frame('UxP_num_of_orders').reset_index()

UxP_features_train['UxP_num_of_orders'] = UxP_num_of_orders(X_products_train)
UxP_features_test['UxP_num_of_orders'] = UxP_num_of_orders(X_products_test)

In [ ]:
#proportion of orders the user purchased the product
#UxP_num_of_orders/U_num_of_orders

UxP_order_ratio_train = pd.merge(UxP_features_train['UxP_num_of_orders'],user_features_train['num_of_orders'],how = 'left',on='user_id')
UxP_order_ratio_train['UxP_order_ratio'] = UxP_order_ratio_train['UxP_num_of_orders']/UxP_order_ratio_train['u_num_of_orders']
UxP_order_ratio_train.drop(['u_num_of_orders','UxP_num_of_orders'],axis=1,inplace=True) 

UxP_order_ratio_test = pd.merge(UxP_features_test['UxP_num_of_orders'],user_features_test['num_of_orders'],how = 'left',on='user_id')
UxP_order_ratio_test['UxP_order_ratio'] = UxP_order_ratio_test['UxP_num_of_orders']/UxP_order_ratio_test['u_num_of_orders']
UxP_order_ratio_test.drop(['u_num_of_orders','UxP_num_of_orders'],axis=1,inplace=True)

UxP_features_train['UxP_order_ratio'] = UxP_order_ratio_train
UxP_features_test['UxP_order_ratio'] = UxP_order_ratio_test

In [ ]:
#number of orders since last purchase of the product
UxP_orders_since_last_order_train = X_products_train.groupby(['user_id'])['order_number'].max()-X_products_train.groupby(['user_id', 'product_id'])['order_number'].max()+1
UxP_orders_since_last_order_test = X_products_test.groupby(['user_id'])['order_number'].max()-X_products_test.groupby(['user_id', 'product_id'])['order_number'].max()+1

UxP_features_train['UxP_orders_since_last'] = UxP_orders_since_last_order_train.to_frame('UxP_orders_since_last').reset_index()
UxP_features_test['UxP_orders_since_last'] = UxP_orders_since_last_order_test.to_frame('UxP_orders_since_last').reset_index()

In [ ]:
#number of orders since last purchase of the product/u_num_of_orders

UxP_orders_since_last_ratio_train = pd.merge(UxP_features_train['UxP_orders_since_last'],user_features_train['num_of_orders'],how = 'left',on='user_id')
UxP_orders_since_last_ratio_train['UxP_orders_since_last_ratio'] = UxP_orders_since_last_ratio_train['UxP_orders_since_last']/UxP_orders_since_last_ratio_train['u_num_of_orders']
UxP_orders_since_last_ratio_train.drop(['u_num_of_orders','UxP_orders_since_last'],axis=1,inplace=True) 

UxP_orders_since_last_ratio_test = pd.merge(UxP_features_test['UxP_orders_since_last'],user_features_test['num_of_orders'],how = 'left',on='user_id')
UxP_orders_since_last_ratio_test['UxP_orders_since_last_ratio'] = UxP_orders_since_last_ratio_test['UxP_orders_since_last']/UxP_orders_since_last_ratio_test['u_num_of_orders']
UxP_orders_since_last_ratio_test.drop(['u_num_of_orders','UxP_orders_since_last'],axis=1,inplace=True) 

UxP_features_train['UxP_orders_since_last_ratio'] = UxP_orders_since_last_ratio_train
UxP_features_test['UxP_orders_since_last_ratio'] = UxP_orders_since_last_ratio_test

In [ ]:
#whether the product was reordered by user before
def UxP_reordered(df):
  return df.groupby(['user_id','product_id'])['reordered'].agg('max').to_frame('UxP_reordered').reset_index()

UxP_reordered_train = UxP_reordered(X_products_train)
UxP_reordered_test = UxP_reordered(X_products_test)

UxP_features_train['UxP_reordered'] = UxP_reordered_train
UxP_features_test['UxP_reordered'] = UxP_reordered_test

In [ ]:
#average order_dow the user purchased the product
def UxP_order_dow_mean(df):
  return df.groupby(['user_id','product_id'])['order_dow'].mean().to_frame('UxP_order_dow_mean').reset_index()

UxP_order_dow_mean_train = UxP_order_dow_mean(X_products_train)
UxP_order_dow_mean_test = UxP_order_dow_mean(X_products_test)

UxP_features_train['UxP_order_dow_mean'] = UxP_order_dow_mean_train
UxP_features_test['UxP_order_dow_mean'] = UxP_order_dow_mean_test

In [ ]:
#UxP_order_dow_std
def UxP_order_dow_std(df):
  return df.groupby(['user_id','product_id'])['order_dow'].std().to_frame('UxP_order_dow_std').reset_index()

UxP_order_dow_std_train = UxP_order_dow_std(X_products_train).fillna(0)
UxP_order_dow_std_test = UxP_order_dow_std(X_products_test).fillna(0)

UxP_features_train['UxP_order_dow_std'] = UxP_order_dow_std_train
UxP_features_test['UxP_order_dow_std'] = UxP_order_dow_std_test

In [ ]:
#UxP_order_hour_of_day_mean
def UxP_order_hour_of_day_mean(df):
  return df.groupby(['user_id','product_id'])['order_hour_of_day'].mean().to_frame('UxP_order_hour_of_day_mean').reset_index()

UxP_order_hour_of_day_mean_train = UxP_order_hour_of_day_mean(X_products_train)
UxP_order_hour_of_day_mean_test = UxP_order_hour_of_day_mean(X_products_test)

UxP_features_train['UxP_order_hour_of_day_mean'] = UxP_order_hour_of_day_mean_train
UxP_features_test['UxP_order_hour_of_day_mean'] = UxP_order_hour_of_day_mean_test

In [ ]:
#UxP_order_hour_of_day_std
def UxP_order_hour_of_day_std(df):
  return df.groupby(['user_id','product_id'])['order_hour_of_day'].std().to_frame('UxP_order_hour_of_day_std').reset_index()

UxP_order_hour_of_day_std_train = UxP_order_hour_of_day_std(X_products_train).fillna(0)
UxP_order_hour_of_day_std_test = UxP_order_hour_of_day_std(X_products_test).fillna(0)

UxP_features_train['UxP_order_hour_of_day_std'] = UxP_order_hour_of_day_std_train
UxP_features_test['UxP_order_hour_of_day_std'] = UxP_order_hour_of_day_std_test

In [ ]:
#UxP_add_to_cart_order_mean
def UxP_add_to_cart_order_mean(df):
  return df.groupby(['user_id','product_id'])['add_to_cart_order'].mean().to_frame('UxP_add_to_cart_order_mean').reset_index()

UxP_add_to_cart_order_mean_train = UxP_add_to_cart_order_mean(X_products_train)
UxP_add_to_cart_order_mean_test = UxP_add_to_cart_order_mean(X_products_test)

UxP_features_train['UxP_add_to_cart_order_mean'] = UxP_add_to_cart_order_mean_train
UxP_features_test['UxP_add_to_cart_order_mean'] = UxP_add_to_cart_order_mean_test

In [ ]:
#UxP_add_to_cart_order_std
def UxP_add_to_cart_order_std(df):
  return df.groupby(['user_id','product_id'])['add_to_cart_order'].std().to_frame('UxP_add_to_cart_order_std').reset_index()

UxP_add_to_cart_order_std_train = UxP_add_to_cart_order_std(X_products_train).fillna(0)
UxP_add_to_cart_order_std_test = UxP_add_to_cart_order_std(X_products_test).fillna(0)

UxP_features_train['UxP_add_to_cart_order_std'] = UxP_add_to_cart_order_std_train
UxP_features_test['UxP_add_to_cart_order_std'] = UxP_add_to_cart_order_std_test

In [ ]:
#UxP last order
def UxP_last_order(df):
  return df.groupby(['user_id','product_id'])['order_number'].max().to_frame('UxP_last_order').reset_index()

UxP_last_order_train = UxP_last_order(X_products_train)
UxP_last_order_test = UxP_last_order(X_products_test)

UxP_features_train['UxP_last_order_train'] = UxP_last_order_train
UxP_features_test['UxP_last_order_test'] = UxP_last_order_test

In [ ]:
#UxP last order ratio

UxP_last_order_ratio_train = pd.merge(UxP_last_order_train,user_features_train['num_of_orders'],how = 'left',on='user_id')
UxP_last_order_ratio_train['UxP_last_order_ratio'] = UxP_last_order_ratio_train['UxP_last_order']/UxP_last_order_ratio_train['u_num_of_orders']
UxP_last_order_ratio_train.drop(['u_num_of_orders','UxP_last_order'],axis=1,inplace=True) 

UxP_last_order_ratio_test = pd.merge(UxP_last_order_test,user_features_test['num_of_orders'],how = 'left',on='user_id')
UxP_last_order_ratio_test['UxP_last_order_ratio'] = UxP_last_order_ratio_test['UxP_last_order']/UxP_last_order_ratio_test['u_num_of_orders']
UxP_last_order_ratio_test.drop(['u_num_of_orders','UxP_last_order'],axis=1,inplace=True)

UxP_features_train['UxP_last_order_ratio'] = UxP_last_order_ratio_train
UxP_features_test['UxP_last_order_ratio'] = UxP_last_order_ratio_test

In [ ]:
df_UxP_features_train = reduce(lambda  left,right: pd.merge(left,right,on=['user_id','product_id'],
                                            how='left'), [feature for key, feature in UxP_features_train.items()])

In [ ]:
df_UxP_features_train.to_csv('UxP_features_train.csv')
!cp UxP_features_train.csv "drive/My Drive//Colab Notebooks/data/"

In [ ]:
df_UxP_features_test = reduce(lambda  left,right: pd.merge(left,right,on=['user_id','product_id'],
                                            how='left'), [feature for key, feature in UxP_features_test.items()])

In [ ]:
df_UxP_features_test.to_csv('UxP_features_test.csv')
!cp UxP_features_test.csv "drive/My Drive//Colab Notebooks/data/"

#last order feature

In [ ]:
last_order_features = orders[orders['eval_set'] == 'train']

df_last_order_train = last_order_features[last_order_features['user_id'].isin(user_id_train)][['user_id','order_dow','order_hour_of_day','days_since_prior_order']]
df_last_order_test = last_order_features[last_order_features['user_id'].isin(user_id_test)][['user_id','order_dow','order_hour_of_day','days_since_prior_order']]

In [ ]:
df_last_order_train.rename(columns={'order_dow':'LO_dow','order_hour_of_day':'LO_hour_of_day','days_since_prior_order':'LO_days_since_prior_order'},inplace=True)
df_last_order_test.rename(columns={'order_dow':'LO_dow','order_hour_of_day':'LO_hour_of_day','days_since_prior_order':'LO_days_since_prior_order'},inplace=True)

In [ ]:
# save to csv and upload to google drive
df_last_order_train.to_csv('last_order_train.csv')
!cp last_order_train.csv "drive/My Drive//Colab Notebooks/data/"

In [ ]:
# save to csv and upload to google drive
df_last_order_test.to_csv('last_order_test.csv')
!cp last_order_test.csv "drive/My Drive//Colab Notebooks/data/"

# **Merge features**

In [ ]:
# merge features for training
df_features_train = pd.merge(df_UxP_features_train,df_user_features_train,how='left',on='user_id')
df_features_train = pd.merge(df_features_train,df_product_features_train,how='left',on='product_id')
df_features_train = pd.merge(df_features_train,df_last_order_train,how='left',on='user_id')

In [ ]:
#merge features for testing
df_features_test = pd.merge(df_UxP_features_test,df_user_features_test,how='left',on='user_id')
df_features_test = pd.merge(df_features_test,df_product_features_test,how='left',on='product_id')
df_features_test = pd.merge(df_features_test,df_last_order_test,how='left',on='user_id')

# **Merge features and target**

In [ ]:
data_train = pd.merge(df_features_train,y_train_merge,how='left',on=['user_id','product_id'])

In [ ]:
data_test = pd.merge(df_features_test,y_test_merge,how='left',on=['user_id','product_id'])

In [ ]:
# save to csv and upload to google drive
data_train.to_csv('data_train.csv')
!cp data_train.csv "drive/My Drive//Colab Notebooks/data/"

In [ ]:
# save to csv and upload to google drive
data_test.to_csv('data_test.csv')
!cp data_test.csv "drive/My Drive//Colab Notebooks/data/"